# Comment/Annotation Nanopublication Generator

This notebook generates paper annotation nanopublications from a JSON configuration file.

## Paper Annotations
These nanopublications capture specific quotations from papers along with the reader's interpretation:
- Exact quotation from the paper (with optional end quote for longer passages)
- Personal interpretation and relevance explanation
- Links back to the source paper via DOI

In [ ]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_uri, make_literal, validate_required_fields,
    PREFIXES
)

In [ ]:
class CommentNanopubGenerator(NanopubGenerator):
    """Generator for paper annotation/comment nanopublications."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': config.get('template_uri'),
            'label': nanopub_config.get('nanopub_label', 'Paper Annotation')
        }
        super().__init__(merged_config)
        self.add_prefix('cito')
    
    def generate_assertion(self) -> str:
        """Generate the comment/annotation assertion graph."""
        paper_doi = self.config['paper_doi']
        quotation = self.config['quotation']
        quotation_end = self.config.get('quotation_end')
        comment = self.config['comment']
        creator_orcid = self.config.get('creator_orcid', '0000-0000-0000-0000')
        
        # Normalize paper DOI to URI
        if not paper_doi.startswith('http'):
            paper_uri = f"https://doi.org/{paper_doi}"
        else:
            paper_uri = paper_doi
        
        lines = [f"{self.sub_prefix}:assertion {{"]
        
        # The creator quotes from the paper
        lines.append(f"  orcid:{creator_orcid} cito:quotes <{paper_uri}> .")
        
        # Add the quoted text
        lines.append(f"  <{paper_uri}> cito:hasQuotedText {make_literal(quotation)} .")
        
        # Add end of quotation if present (for longer passages)
        if quotation_end:
            lines.append(f"  <{paper_uri}> cito:hasQuotedTextEnd {make_literal(quotation_end)} .")
        
        # Add the comment/interpretation
        lines.append(f"  <{paper_uri}> rdfs:comment {make_literal(comment)} .")
        
        lines.append("}")
        return "\n".join(lines)

In [ ]:
# Configuration
CONFIG_FILE = "../config/vbae208_comment.json"  # Change this to use different config
OUTPUT_DIR = "../output/comment"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"DOI: {config['metadata']['source_paper']['doi']}")
print(f"Number of annotation nanopublications to generate: {len(config['nanopublications'])}")
print()

for i, np_config in enumerate(config['nanopublications'], 1):
    print(f"{i}. Quotation: \"{np_config['quotation'][:60]}...\"")
    print(f"   Comment: {np_config['comment'][:60]}...")
    print()

In [ ]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = CommentNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

In [ ]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

## Next Steps

1. Review the generated `.trig` files in the output directory
2. Sign and publish using Nanodash or nanopub-java
3. To use with a different paper, create a new JSON config file and update `CONFIG_FILE`